# Long candle strategy  EDA

Long candle strategy is classification model.
Features: prices
Targets: -1,0,1 signal

To install conda and binance packages to this notebook uncomment the code below

In [1]:
%%capture
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install pandas
#%pip install huobi-sdk==2.3.3

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
#mode="dev" # dev - work with local dev data
mode="yc" # yc - work with data from yandex cloud bots

symbol="BTC-USDT"
strategy="LongCandleDenseStrategy"

In [3]:

import os

# Downloading latest pytrade2 data
if mode == "yc":
    os.system("cd ./../deploy/yandex_cloud; ./download_data.sh")
    data_dir=f"../data/yandex-cloud"
else:
    data_dir=f"../data/dev"
    
print(f"Download completed. Local data dir: {data_dir}")

    

deploy_lib.sh: line 4: yc: command not found
INFO: Retrieving list of remote files for s3://pytrade2/data/ ...


Sync pytrade2 data from s3://pytrade2/data to /home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud


INFO: No cache file found, creating it.
INFO: Compiling list of local files...
INFO: Running stat() and reading/calculating MD5 values on 1702 files, this may take some time...
INFO: [1000/1702]
INFO: Found 32 remote files, 1702 local files
INFO: Verifying attributes...
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-07_BTC-USDT_x.csv
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-08_BTC-USDT_x.csv
INFO: disabled md5 check for LongCandleDenseStrategy/Xy/2023-10-09_BTC-USDT_x.csv
INFO: Summary: 10 remote files to download, 1675 local files to delete, 0 local files to hardlink


download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_learn_x.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_learn_x.csv'  [1 of 10]
 938969 of 938969   100% in    0s     2.61 MB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_learn_y.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_learn_y.csv'  [2 of 10]
 37065 of 37065   100% in    0s   573.42 kB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_x.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_x.csv'  [3 of 10]
 5456657 of 5456657   100% in    1s     5.13 MB/s  done
download: 's3://pytrade2/data/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_y.csv' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud

## Predicted and actual low/high

In [11]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_last(symbol: str, name: str, n=1000):
    xy_dir=f"{data_dir}/{strategy}/Xy"

    file_path = sorted([f for f in os.listdir(xy_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
    file_path = f"{xy_dir}/{file_path}"
    print(f"Read from {file_path}")
    return pd.read_csv(file_path, parse_dates=True).tail(n)


def read_last_xy(data_dir):
    """ Read days from start to end from local predictions history"""
    
    #data=read_last(symbol, "data", n)
    X=read_last(symbol, "x")
    y=read_last(symbol, "y")
    data=pd.merge(X,y, left_index=True, right_index=True) 
    #y=pd.concat([data,y])
    return data,X,y


# Read candles for last 3 days
print(symbol)
data,x,y = read_last_xy(data_dir)
y.tail()


BTC-USDT
Read from ../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_x.csv
Read from ../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_y.csv


,Unnamed: 0,signal_pred,signal_target
8396,2023-10-14 23:41:00.006,0,0
8397,2023-10-14 23:41:29.258,0,0
8398,2023-10-14 23:42:00.028,0,0
8399,2023-10-14 23:43:00.004,0,0
8400,2023-10-14 23:43:02.421,0,0


In [18]:
print(read_last(symbol, 'learn_y').columns)

Read from ../data/yandex-cloud/LongCandleDenseStrategy/Xy/2023-10-14_BTC-USDT_learn_y.csv
Index(['2023-10-14 00:00:05.812', '0'], dtype='object')


signal_pred
0    1000
Name: count, dtype: int64